# Module 4. Personalize 캠페인과 실시간 상호 작용 하기

이 노트북은 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 탐색하는 동안 사용자의 의도가 변경되면, 해당 동작에 따라 수정된 추천 영화 목록들이 표시됩니다.

또한 추천 결과가 반환되기 전, 영화를 선택하는 사용자 행동을 시뮬레이션하기 위한 데모 코드를 보여줍니다.

우선, Personalize에 필요한 라이브러리를 가져 오는 것부터 시작합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [2]:
%store -r


생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [3]:
suffix = str(np.random.uniform())[4:9]

In [4]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 이벤트 추적기 생성

추천 시스템이 실시간 이벤트에 응답하기 전에 이벤트 추적기(Event Tracker)가 필요합니다. 아래 코드 셀에서 이벤트 추적기 하나를 생성하고 이 실습에서 계속 사용하겠습니다. 이벤트 추적기 이름을 `MovieClickTracker`로 부여하였지만, 여러분들이 자유롭게 지정하실 수 있습니다.

In [5]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:ap-northeast-2:870180618679:event-tracker/5a5265bc
19fab5ff-dbe8-43bd-aaca-14481bd2ec28


In [6]:
event_tracker_arn = response['eventTrackerArn']

## 소스 데이터 설정

위의 추적 ID가 표시되며, 이 ID는 변수에 할당되었으므로 추가 조치가 필요하지 않습니다. 아래 코드 셀을 실행하면 나중에 추천 영화 목록을 출력할 수 있도록 합니다.

In [7]:
# First load items into memory
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()
items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']

print(items.head())
print(items.tail())

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

get_movie_title(3952)


   ITEM_ID                               TITLE                         GENRE
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
      ITEM_ID                       TITLE           GENRE
3878     3948     Meet the Parents (2000)          Comedy
3879     3949  Requiem for a Dream (2000)           Drama
3880     3950            Tigerland (2000)           Drama
3881     3951     Two Family House (2000)           Drama
3882     3952       Contender, The (2000)  Drama|Thriller


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


['Contender, The (2000)']

## 추천 결과 확인

먼저 이전 노트북에서 생성했던 추천 영화 데이터프레임을 다시 확인합니다.

In [8]:
recommendations_df

,OriginalRecs
0,"Slums of Beverly Hills, The (1998)"
1,Singles (1992)
2,So I Married an Axe Murderer (1993)
3,Notting Hill (1999)
4,"House of Yes, The (1997)"
5,Unstrung Heroes (1995)
6,"Wedding Singer, The (1998)"
7,Manhattan Murder Mystery (1993)
8,Drop Dead Gorgeous (1999)
9,Bob Roberts (1992)


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

In [9]:
session_dict = {}

In [10]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
   
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    "eventValue": float(4)
    }
    event_json = json.dumps(event)
    
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID, # 이벤트트래커에서 생성한 아이디
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'RATING',
        'properties': event_json
        }]
)
    
    
    

아래 코드 셀에서 사용자가 특정 제목을 클릭한 것처럼 이벤트 추적기가 업데이트됩니다.

아래 셀에서 생성된 테이블이 추천 결과들을 이동시키지 않으면 위 셀에서 다른 임의의 3자리 숫자를 시도하고 두 셀을 모두 다시 실행해 주세요. 
여러분은 세 번째 열에서 추천 결과를 확인할 수 있습니다.

In [11]:
# Pick a movie, we will use ID 270 or Miracle on 34th Street
movie_to_click = 3952
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

In [12]:
user_id, movie_to_click, movie_title_clicked

(1749, 3952, ['Contender, The (2000)'])

이 블록을 실행한 후에는 이벤트 추적이 활성화되어 있고 이벤트를 personalize 서비스로 보낸 추천 영화들의 변경 사항이 표시됩니다.

In [13]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  1749


,OriginalRecs,"(Contender, The (2000),)"
0,"Slums of Beverly Hills, The (1998)","Slums of Beverly Hills, The (1998)"
1,Singles (1992),Singles (1992)
2,So I Married an Axe Murderer (1993),So I Married an Axe Murderer (1993)
3,Notting Hill (1999),Notting Hill (1999)
4,"House of Yes, The (1997)","House of Yes, The (1997)"
5,Unstrung Heroes (1995),Unstrung Heroes (1995)
6,"Wedding Singer, The (1998)","Wedding Singer, The (1998)"
7,Manhattan Murder Mystery (1993),Manhattan Murder Mystery (1993)
8,Drop Dead Gorgeous (1999),Drop Dead Gorgeous (1999)
9,Bob Roberts (1992),Bob Roberts (1992)


## 결론

사용자가 상호 작용하는 영화를 변경하여 추천 사항이 변경되었음을 알 수 있습니다. 이 시스템은 사용자가 item 모음과 상호 작용하는 모든 응용 프로그램으로 확장할 수 있습니다. 이러한 도구는 언제든지 사용 가능한 데이터로 가능한 것을 풀다운하여 탐색할 수 있습니다.

Cleanup 노트북에 필요한 값을 저장하려면 아래의 셀을 실행해 주세요.

마지막으로 계정에서 본 핸즈온에서 생성했던 리소스들을 제거할 준비가 되면 `Cleanup.ipynb` 노트북을 열고 코드 셀들을 실행해 주세요.

In [14]:
%store event_tracker_arn

Stored 'event_tracker_arn' (str)
